## <font color = 'maroon'>Assignment 2, part 1 of 2</font>

Questions 1 is a pen-and-paper exercise; questions 2-3 are coding exercises.

Please **show your work** - i.e., show and explain your code/math, and write your reasoning.

1. A data warehouse for a music streaming company consists of the dimensions song, user, time (time and date of when the user listened to a song), and a measure called fee (fee paid by the streaming company to the artist every time a user listens to that song). 
    1. **[1 mark]** Draw a schema diagram for the above data warehouse using a star schema.
    2. **[0.5 marks]** Starting with the base cuboid [time, user, song], what specific OLAP operations should be performed in order to list the total fee collected for a given song for a given month of a given year (e.g. October 2021)?
    3. **[0.5 marks]** Assume that the time dimension has 3 levels: day, month, year; and that the song and user dimensions both have 1 level (not including the virtual level 'all'). How many cuboids will this cube contain (including the base and apex cuboids)?

1. **[1 mark]** Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your work in the PDF report. You can read the [Cubes package documentation](https://readthedocs.org/projects/cubes/downloads/pdf/latest/) for assistance in this task.

1. Consider the CSV file "country-income-large.csv":
   1. **[1 marks]** Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files you have created.
   2. **[1 mark]** Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 30 and 50.
  

In [ ]:
# If you are using Google Colab, you would need to run the lines below to install Cubes. 
# This might also be the case if you are running the notebook locally.
# You can comment the below line if you are running a local python installation with Cubes installed.
# The below line also installs a particular version of the sqlalchemy package needed to use Cubes.

#!pip install cubes
#!pip install -Iv sqlalchemy==1.3.9

from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

In [ ]:
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "country-income-large.csv",
                      table_name="country_income",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("online_shopper", "string"),
                      ],
                      create_id=True
                     )

In [ ]:
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [ ]:
workspace.list_cubes()

[]

[1 mark] Using the same CSV file and data cube in the above lab tutorial, modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year. Make sure to show your work in the PDF report. You can read the Cubes package documentation for assistance in this task.



In [ ]:
workspace.import_model("tutorial_model.json")

In [ ]:
cube = workspace.cube("ibrd_balance")

In [ ]:
browser = workspace.browser(cube)

In [ ]:
result = browser.aggregate(drilldown=["year"])
for record in result:
    print(record)

{'year': 2009, 'amount_sum': 550840, 'record_count': 31, 'amount_min': -1683, 'amount_max': 110040}
{'year': 2010, 'amount_sum': 566020, 'record_count': 31, 'amount_min': -3043, 'amount_max': 128577}


Consider the CSV file "country-income-large.csv":

[1 marks] Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files you have created.

[1 mark] Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 30 and 50.

In [ ]:
workspace.import_model("country_income.json")

In [ ]:
income_cube = workspace.cube("country_income")

In [ ]:
income_browser = workspace.browser(income_cube)

In [ ]:
result = income_browser.aggregate()
print(result.summary)

{'income_sum': 5159200, 'income_avg': 85986.66666666667, 'income_min': 31000, 'income_max': 143000}


In [ ]:
result = income_browser.aggregate(drilldown=["region"])
for record in result:
    print(record)

{'region': 'Brazil', 'income_sum': 1198200, 'income_avg': 79880.0, 'income_min': 31000, 'income_max': 135000}
{'region': 'India', 'income_sum': 1954200, 'income_avg': 84965.21739130435, 'income_min': 35000, 'income_max': 141000}
{'region': 'USA', 'income_sum': 2006800, 'income_avg': 91218.18181818182, 'income_min': 45000, 'income_max': 143000}


In [ ]:
result = income_browser.aggregate(drilldown=["online_shopper"])
for record in result:
    print(record)

{'online_shopper': 'No', 'income_sum': 3150400, 'income_avg': 87511.11111111111, 'income_min': 31000, 'income_max': 143000}
{'online_shopper': 'Yes', 'income_sum': 2008800, 'income_avg': 83700.0, 'income_min': 41000, 'income_max': 139000}


In [ ]:
import cubes as cubes
cuts = [cubes.RangeCut("age", [30], [50])]
cell = cubes.Cell(income_cube, cuts)
result = income_browser.aggregate(cell, drilldown=["age"])
for record in result:
    print(record)

{'age': 30, 'income_sum': 59000, 'income_avg': 59000.0, 'income_min': 59000, 'income_max': 59000}
{'age': 32, 'income_sum': 57600, 'income_avg': 57600.0, 'income_min': 57600, 'income_max': 57600}
{'age': 34, 'income_sum': 58000, 'income_avg': 58000.0, 'income_min': 58000, 'income_max': 58000}
{'age': 35, 'income_sum': 120800, 'income_avg': 60400.0, 'income_min': 56000, 'income_max': 64800}
{'age': 36, 'income_sum': 70000, 'income_avg': 70000.0, 'income_min': 70000, 'income_max': 70000}
{'age': 39, 'income_sum': 71000, 'income_avg': 71000.0, 'income_min': 71000, 'income_max': 71000}
{'age': 40, 'income_sum': 69600, 'income_avg': 69600.0, 'income_min': 69600, 'income_max': 69600}
{'age': 41, 'income_sum': 64000, 'income_avg': 64000.0, 'income_min': 64000, 'income_max': 64000}
{'age': 42, 'income_sum': 229400, 'income_avg': 76466.66666666667, 'income_min': 72000, 'income_max': 80400}
{'age': 43, 'income_sum': 297200, 'income_avg': 74300.0, 'income_min': 69000, 'income_max': 79000}
{'age':